In [22]:
import os
import json
import openai

if not os.environ.get("OPENAI_API_KEY"):
    print("OPENAI_API_KEY isn't set; set it in your environment (and restart the noteboook) or set it below...")


In [23]:
# If you didn't set the key...
# os.environ["OPENAI_API_KEY"] = "sk-..."
client = openai.OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)


In [24]:
transcript_directory = os.path.abspath("../transcripts")
transcript_filenames = os.listdir(transcript_directory)
print(transcript_directory, transcript_filenames)
link_data = json.loads(open("../links.json").read())

/Users/ianbicking/src/podcast-tools/transcripts ['episode-4-18.txt', 'episode-4-17.txt', 'episode-4-16.txt', 'episode-4-14.txt', 'episode-4-15.txt']


In [27]:
transcripts = {}
links = {}
for fn in transcript_filenames:
    transcripts[fn] = open(os.path.join(transcript_directory, fn)).read()
    links[fn] = [item for item in link_data if item["name"] == fn][0]["url"]
for key, value in transcripts.items():
    bytes = len(value)
    words = len(value.split())
    print(f"{key}: {bytes} bytes, {words} words")


episode-4-18.txt: 50455 bytes, 9270 words
episode-4-17.txt: 41242 bytes, 7529 words
episode-4-16.txt: 36730 bytes, 6435 words
episode-4-14.txt: 55325 bytes, 10177 words
episode-4-15.txt: 45859 bytes, 8243 words


In [28]:
transcript_prompt = """
You are given a raw podcast transcript. **Transform it into Markdown** by following these rules:

1. **Remove or Merge Filler Words**
   - Delete common fillers and repeated words like "um," "uh," "you know," "like," "right?," etc. unless needed for meaning.
   - Merge short interjections (e.g., “Yeah,” “Sure”) into the nearest sentence if they don’t add standalone value.

   **Examples**:
   ```
   INPUT:
   [00:02:10] Justin Grammens: Um, I'm not sure, but, like, we can definitely talk about XR, right?

   OUTPUT:
   [00:02:10] Justin Grammens: I'm not sure, but we can definitely talk about [XR (extended reality)](https://www.google.com/search?q=XR+extended+reality).
   ```

   ```
   INPUT:
   [00:03:00] Nick Roseth: Yeah. Yeah. So, I was thinking, um, that it might be easier just to, you know, do it ourselves.

   OUTPUT:
   [00:03:00] Nick Roseth: I was thinking it might be easier to do it ourselves.
   ```

2. **Link References and Acronyms**
   - Use Markdown links for references (books, studies, organizations) with a placeholder Google search URL.
   - For first mention of an acronym, include an expansion in parentheses, then link it.

   **Examples**:
   ```
   INPUT:
   [00:04:05] Justin Grammens: There's a PWC study about adoption.

   OUTPUT:
   [00:04:05] Justin Grammens: There's a [PWC study](https://www.google.com/search?q=PWC+study+about+adoption) about adoption.
   ```

   ```
   INPUT:
   [00:04:22] Nick Roseth: We used RAG or retrieval augmented generation.

   OUTPUT:
   [00:04:22] Nick Roseth: We used [RAG (retrieval augmented generation)](https://www.google.com/search?q=RAG+retrieval+augmented+generation).
   ```

3. **Create Lists Where Appropriate**
   - When speakers enumerate items (e.g. “first, second, third”), convert them into a Markdown list.

   **Example**:
   ```
   INPUT:
   [00:05:10] Justin Grammens: We have three steps. One, gather data. Two, build a model. Three, evaluate results.

   OUTPUT:
   [00:05:10] Justin Grammens:
   We have three steps:
   1. Gather data
   2. Build a model
   3. Evaluate results
   ```

4. **Intro and Outro in Blockquotes**
   - If you detect content that introduces or concludes the show (like sponsor ads, host signoffs), enclose that text in blockquotes (`>`). Each line should start with `>`.

   **Example**:
   ```
   INPUT:
   [00:00:38] AI Voice: You can visit us at AppliedAI.mn to keep up to date on events. Thanks for listening.

   OUTPUT:
   > You can visit us at AppliedAI.mn to keep up to date on events.
   > Thanks for listening.
   ```

   Any portion of the transcript read by a "AI Announcer" is an intro/outro. The speaker can be removed and the text blockquoted.

5. **Use Paragraph Breaks**
   - Break large blocks of speech into paragraphs of ~3–5 sentences (or ~250 words) each.
   - Start a new paragraph at a logical topic shift.

   **Example**:
   ```
   INPUT:
   [00:06:40] Nick Roseth: So I started in design years ago, focusing on print, then moved to web, then eventually found AR and VR. I realized those technologies can transform industries, from construction to healthcare, but there's slow adoption. The reality is, we need to examine human behavior and acceptance of emerging tech. It's not just about the technology. A big factor is trust.

   OUTPUT:
   [00:06:40] Nick Roseth:
   I started in design years ago, focusing first on print, then moving into web. Eventually, I found AR and VR. I realized these technologies can transform industries, from construction to healthcare.

   There's still slow adoption, because we have to look at human behavior and acceptance of emerging tech. It's not just about the technology itself. A big factor is trust.
   ```

6. **No Headings**
   - Do not insert or create headings (e.g., `# Introduction`, `## Overview`, etc.).
   - Stick to paragraphs, blockquotes, links, and lists only.

7. **Keep Speaker Labels on One Line**
   - Format each speaker turn as `"[timestamp] Speaker: text"`.  (Omit the timestamp if not given)
   - Use the exact speaker name from the transcript (no custom headings).
   - **Example**:
     ```
     [00:07:30] Justin Grammens:
     Let's talk about data privacy next.

     becomes

     [00:07:30] Justin Grammens: Let's talk about data privacy next.
     ```

8. **Timestamps**
   - Retain original timestamps in the format `[00:00]`.
   - Keep them at the start of each speaker line, directly before the speaker’s name.
   - Do not remove, alter, or relocate them.

---

**Return the cleaned transcript in Markdown** following these rules. Remember:

- **Remove filler words**
- **Link references/acronyms**
- **Create lists** (if items are enumerated)
- **Blockquote** intro/outro text
- **Insert paragraph breaks** for readability
- **No headings**
- **Speaker label on one line**
- **Keep timestamps**

Use these **expanded examples** as a guide:

```
INPUT:
[00:10:05] Justin Grammens: Um, so I think we have, like, a few new developments. One, the Apple Vision Pro. Two, new updates in ChatGPT. Three, expansions in RAG, right?

OUTPUT:
[00:10:05] Justin Grammens:
I think we have a few new developments:
1. The [Apple Vision Pro](https://www.google.com/search?q=Apple+Vision+Pro)
2. New updates in [ChatGPT](https://www.google.com/search?q=ChatGPT)
3. Expansions in [RAG (retrieval augmented generation)](https://www.google.com/search?q=RAG+retrieval+augmented+generation)
```

```
INPUT:
[00:12:45] AI Voice: Thank you for listening to the show. Please visit us at OurSite.com.

OUTPUT:
> Thank you for listening to the show.
> Please visit us at [OurSite.com](https://www.google.com/search?q=OurSite.com)
```

```
INPUT:
[00:15:33] Nick Roseth: Right, yeah, so, um, I guess XR is basically extended reality, so, yeah, I guess that's it.

OUTPUT:
[00:15:33] Nick Roseth: XR is basically [extended reality](https://www.google.com/search?q=extended+reality).
```
"""

In [29]:
def rewrite_transcript(text):
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": transcript_prompt},
            {"role": "user", "content": text},
        ]
    )
    return resp.choices[0].message.content


In [30]:
def strip_quotes(text):
    text = text.strip()
    if text.startswith("```"):
        text = text.split("\n", 1)[1]
    if text.endswith("```"):
        text = text.rsplit("\n", 1)[0]
    return text.strip()

In [33]:
example_name = list(transcripts.keys())[0]
revised = rewrite_transcript(transcripts[example_name])
print(strip_quotes(revised))

[00:00:00] David Wynn: The mental model I like to give people is imagine a guy in a bar overheard 10,000 hours of two people next to him talking about motorcycles. He has never seen, driven, heard, or felt one. When you ask him a question about motorcycles, he’ll probably give you the right answer. Occasionally, he might compliment how your torque smells. It's not his fault. He's just trying to piece together everything he could possibly know from what he's heard. He's not really thinking it through. I think that model is key for people to grasp. As a result, summarizing—where I'm trusting someone to come up with the important stuff I need to know—feels a bit suspicious to me. If we called it shortening, that’d be different. Summarizing, I’m a little wary of.

[00:00:48] AI Announcer: Welcome to the Conversations on Applied AI podcast, where Justin Grammens and the team at Emerging Technologies North talk with experts in the fields of artificial intelligence and deep learning. In each 

In [34]:
from IPython.display import Markdown, display
display(Markdown(strip_quotes(revised)))

[00:00:00] David Wynn: The mental model I like to give people is imagine a guy in a bar overheard 10,000 hours of two people next to him talking about motorcycles. He has never seen, driven, heard, or felt one. When you ask him a question about motorcycles, he’ll probably give you the right answer. Occasionally, he might compliment how your torque smells. It's not his fault. He's just trying to piece together everything he could possibly know from what he's heard. He's not really thinking it through. I think that model is key for people to grasp. As a result, summarizing—where I'm trusting someone to come up with the important stuff I need to know—feels a bit suspicious to me. If we called it shortening, that’d be different. Summarizing, I’m a little wary of.

[00:00:48] AI Announcer: Welcome to the Conversations on Applied AI podcast, where Justin Grammens and the team at Emerging Technologies North talk with experts in the fields of artificial intelligence and deep learning. In each episode, we cut through the hype and dive into how these technologies are being applied to real-world problems today. We hope you find this episode educational and applicable to your industry. Connect with us to learn more about our organization at [AppliedAI.mn](https://www.google.com/search?q=AppliedAI.mn). Enjoy.

[00:01:19] Justin Grammens: Welcome, everyone, to the Conversations on Applied AI podcast. I'm your host, Justin Grammens, and our guest today is David Wynn. David is the Principal Solution Architect at EdgeDelta and an enthusiast for all things technology. 

With a career spanning 15 years, including a pivotal role at Google Cloud for Games, David has cultivated a deep understanding of technical sales, solution architecture, and the transformative power of cloud services in the gaming industry and beyond. His unique perspective challenges the overreliance on AI, advocating for a balanced approach that leverages human ingenuity alongside technological advancements, especially in areas like observability.

When he's not steering innovation, David revels in the vibrant geek culture of Atlanta and is a regular at DragonCon, the largest multimedia popular culture convention focusing on science fiction and fantasy. His diverse interests span from technology to philosophy. I’m sure we’re going to have an awesome conversation today about AI and its applications. Thank you so much for being on the program.

[00:02:11] David Wynn: Absolutely, Justin. Pleasure to be here.

[00:02:13] Justin Grammens: Awesome. I touched on some of your background at Google and your current role. I'm curious, how did you get into technology? Were you always fascinated with programming?

[00:02:31] David Wynn: I grew up in Greenville, South Carolina, which is not a technical hub in the U.S. My only teacher, Miss Harmon, was my computer science teacher throughout high school. That was just enough to spark my interest, but I went to college for economics instead. I had the wrong impression about what it meant to be a programmer, so I thought I’d do something else.

After college, I ended up spending time at UPS during the 2008 crash, where a PhD and I worked on econometrics to improve forecasting. My role involved a lot of VBA code to make Excel work for 250 time series data sets, which is where I started developing practical skills. 

Later, I moved to California for a relationship, and at the Stanford career fair, I found a lot of technology opportunities. I ended up doing post-sales implementation middleware, which was a form of ETL. I built scalable data pipelines for law firms and transitioned into observability at Sumo Logic. Eventually, I joined Google Cloud when it was small, around 70 sales engineers globally, which was a rapid learning experience for cloud technologies. Now, I'm back in observability at Edge Delta.

[00:05:42] Justin Grammens: Great journey! For our audience, could you define what observability is?

[00:05:51] David Wynn: Depending on how good your Googling or using GPT is, observability can be defined simply as logs, metrics, and traces. Essentially, an application generates data about itself to show how well it's running. Collecting and displaying that information is observability.

A broader way to define it is the process of ensuring that what you’re doing is operating as expected. This understanding goes beyond any specific metric, allowing for flexibility in approach. The observability space is transitioning because our data has become enormous, reaching hundreds of terabytes to petabytes generated daily.

When running applications on multiple machines, observability software pulls all that data together to find what’s important. However, scalability has become an issue as the volume increases.

[00:07:24] Justin Grammens: I've worked in the Internet of Things and sensors for over 15 years. At some point, processing becomes overwhelming, and I turned to machine learning and deep learning to filter through the noise. Can you discuss AI's role in observability?

[00:07:54] David Wynn: There’s a group of people very hopeful about AI, believing that if we throw all our data into it, it will find solutions. On the other hand, we know that errors and crashes are bad, and we generally understand the data we have. For example, at Edge Delta, we use a traditional machine learning model to identify statistical abnormalities. Then, we feed this information into an LLM to suggest next actions to take.

This is like having a 2 a.m. crib sheet for trouble-shooting—a quick list of things to check if something goes wrong. This process can be very useful for resolving issues and getting people back to bed, especially for those new to the team.

[00:09:27] Justin Grammens: It's a practical application of generative AI, particularly for junior engineers on call. They need backup when systems fail at inconvenient times.

[00:09:50] David Wynn: Exactly. When debugging, you may be isolating problems across teams and encountering new challenges. This complexity can vary greatly if there’s a lack of communication around team decision-making and implementation.

[00:10:21] Justin Grammens: I've talked to engineers who have difficulty deciphering logs and hardware dumps. They're looking for anomaly detection to guide them when they aren't familiar with the code or libraries.

[00:10:50] David Wynn: Different approaches exist to abstract problems. Some errors may require immediate resolution, while other conditions involve deeper awareness of a broader context. Higher-level issues may relate to architecture state or model performance.

LLMs can help translate command logs into actionable insights, but they struggle with new or unprecedented problems. For instance, I encountered a unique billing error at Intap—a failure in our sync service due to an unpaid account. LLMs might not provide guidance on rare errors that haven’t been encountered before, but they excel in generating ideas and constructing outlines for broader queries.

[00:13:55] Justin Grammens: I agree that LLMs lack intuition when it comes to errors. They're great for specific tasks, but they struggle with nuances and unique scenarios. And while they improve efficiency in mundane tasks, they're not perfect tools for summarizing or assessing the uniqueness of issues.

[00:15:47] David Wynn: Absolutely. While they can help streamline processes, their ability to resolve complex problems is limited.

[00:15:59] Justin Grammens: There’s concern around AI potentially replacing jobs. What’s your view on that?

[00:16:01] David Wynn: I’m generally not worried. If you try to get these technologies to perform tasks you’re concerned about, you might find challenges. For example, creating sophisticated art with AI can often lead to frustrating results. People will always seek specificity and control, which can be lost with AI.

My impression is that AI is not going to completely replace roles but will enhance processes, helping eliminate inefficient tasks. AI will streamline operations, allowing people to focus on higher-level problem-solving.

[00:18:53] Justin Grammens: Makes sense. Current tools enhance capabilities rather than completely displace them. People will experience increased ROI with smart tools. 

[00:20:26] David Wynn: Exactly! Many current AI tools become more intelligent through user interaction, enhancing daily workflows.

[00:20:40] Justin Grammens: While generative AI has captured imaginations, many startups will fade away amidst the hype. Existing products will adapt and integrate AI efficiently.

[00:21:44] David Wynn: Agreed! Many companies are focusing on value-driven models rather than sheer size. Innovations will improve specific applications, creating more effective and targeted solutions.

[00:22:59] Justin Grammens: Observability is key to identifying and fixing system issues, as we saw with CrowdStrike recently. It highlights the need for better testing and pre-release verification of system changes.

[00:27:52] David Wynn: Absolutely! If a deployment has a large blast radius, precautions should have been taken to catch potential issues. Observability measures must be effective enough to predict problems before they occur.

[00:30:50] Justin Grammens: Drawing parallels with industry history—confidence can lead to overlooking vital checks.  Just as NASA faced challenges due to rapid launches, companies risk similar pitfalls without due diligence.

[00:31:39] David Wynn: Yes, we must prevent overconfidence in testing and deploy agile systems that acknowledge potential issues.

[00:34:08] Justin Grammens: Many companies fail to employ chaos engineering, disregarding the value of stress-testing systems to ensure resilience.

[00:34:24] David Wynn: Indeed! Chaos engineering forces an organization to consider the worst-case scenarios actively while addressing system performance expectations.

[00:34:53] Justin Grammens: As we wrap up, what advice would you give someone entering the technology field?

[00:35:04] David Wynn: My advice is to follow your interests. The niche topics you’re drawn to may lead to innovative ideas. You can find resources on almost anything today. Engage with communities, ask questions online, fail, and learn—it's all part of the process!

[00:36:53] Justin Grammens: Great advice. Even older resources can provide valuable insights, as industries often evolve at a slower pace.

[00:39:12] David Wynn: Exactly! It’s about finding that intersection of interests and skills.

[00:40:00] Justin Grammens: How can people connect with you after this conversation?

[00:40:21] David Wynn: LinkedIn is best for reaching me. I have a professional-looking blazer in my profile picture!

[00:40:47] Justin Grammens: We'll have your LinkedIn page and the relevant links in our show notes. Is there anything else you'd like to share?

[00:41:58] David Wynn: I want to leave everyone with a hopeful thought. Many of us feel overwhelmed by the speed of change, thinking we don’t matter. Remember, the giants we stand on had the same doubts. They created impactful things, and so can you.

[00:42:45] Justin Grammens: Inspiring indeed, David! It’s been a pleasure having you on the show. We appreciate your time and insights. Wishing you the best at Edge Delta and your future endeavors.

[00:42:49] AI Announcer: You've listened to another episode of the Conversations on Applied AI podcast. We hope you are eager to learn more about applying artificial intelligence and deep learning within your organization. You can visit us at [AppliedAI.mn](https://www.google.com/search?q=AppliedAI.mn) to keep up to date on our events and connect with our amazing community. Please don't hesitate to reach out to Justin at [AppliedAI.mn](https://www.google.com/search?q=AppliedAI.mn) if you are interested in participating in a future episode. Thank you for listening.

In [35]:
import re

def convert_timestamps_to_links(text, base_url):
    def replace_timestamp(match):
        time_str = match.group(1)
        parts = time_str.split(":")
        if len(parts) == 3:  # hh:mm:ss
            h, m, s = parts
            total_seconds = int(h) * 3600 + int(m) * 60 + int(s)
        elif len(parts) == 2:  # mm:ss
            m, s = parts
            total_seconds = int(m) * 60 + int(s)
        else:  # fallback
            total_seconds = int(parts[0])
        return f"[{time_str}]({base_url}?t={total_seconds})"

    pattern = r"\[([0-9]{1,2}:[0-9]{2}(?::[0-9]{2})?)\]"
    return re.sub(pattern, replace_timestamp, text)


In [36]:
linked_markdown = convert_timestamps_to_links(strip_quotes(revised), links[example_name])
display(Markdown(linked_markdown))

[00:00:00](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=0) David Wynn: The mental model I like to give people is imagine a guy in a bar overheard 10,000 hours of two people next to him talking about motorcycles. He has never seen, driven, heard, or felt one. When you ask him a question about motorcycles, he’ll probably give you the right answer. Occasionally, he might compliment how your torque smells. It's not his fault. He's just trying to piece together everything he could possibly know from what he's heard. He's not really thinking it through. I think that model is key for people to grasp. As a result, summarizing—where I'm trusting someone to come up with the important stuff I need to know—feels a bit suspicious to me. If we called it shortening, that’d be different. Summarizing, I’m a little wary of.

[00:00:48](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=48) AI Announcer: Welcome to the Conversations on Applied AI podcast, where Justin Grammens and the team at Emerging Technologies North talk with experts in the fields of artificial intelligence and deep learning. In each episode, we cut through the hype and dive into how these technologies are being applied to real-world problems today. We hope you find this episode educational and applicable to your industry. Connect with us to learn more about our organization at [AppliedAI.mn](https://www.google.com/search?q=AppliedAI.mn). Enjoy.

[00:01:19](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=79) Justin Grammens: Welcome, everyone, to the Conversations on Applied AI podcast. I'm your host, Justin Grammens, and our guest today is David Wynn. David is the Principal Solution Architect at EdgeDelta and an enthusiast for all things technology. 

With a career spanning 15 years, including a pivotal role at Google Cloud for Games, David has cultivated a deep understanding of technical sales, solution architecture, and the transformative power of cloud services in the gaming industry and beyond. His unique perspective challenges the overreliance on AI, advocating for a balanced approach that leverages human ingenuity alongside technological advancements, especially in areas like observability.

When he's not steering innovation, David revels in the vibrant geek culture of Atlanta and is a regular at DragonCon, the largest multimedia popular culture convention focusing on science fiction and fantasy. His diverse interests span from technology to philosophy. I’m sure we’re going to have an awesome conversation today about AI and its applications. Thank you so much for being on the program.

[00:02:11](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=131) David Wynn: Absolutely, Justin. Pleasure to be here.

[00:02:13](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=133) Justin Grammens: Awesome. I touched on some of your background at Google and your current role. I'm curious, how did you get into technology? Were you always fascinated with programming?

[00:02:31](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=151) David Wynn: I grew up in Greenville, South Carolina, which is not a technical hub in the U.S. My only teacher, Miss Harmon, was my computer science teacher throughout high school. That was just enough to spark my interest, but I went to college for economics instead. I had the wrong impression about what it meant to be a programmer, so I thought I’d do something else.

After college, I ended up spending time at UPS during the 2008 crash, where a PhD and I worked on econometrics to improve forecasting. My role involved a lot of VBA code to make Excel work for 250 time series data sets, which is where I started developing practical skills. 

Later, I moved to California for a relationship, and at the Stanford career fair, I found a lot of technology opportunities. I ended up doing post-sales implementation middleware, which was a form of ETL. I built scalable data pipelines for law firms and transitioned into observability at Sumo Logic. Eventually, I joined Google Cloud when it was small, around 70 sales engineers globally, which was a rapid learning experience for cloud technologies. Now, I'm back in observability at Edge Delta.

[00:05:42](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=342) Justin Grammens: Great journey! For our audience, could you define what observability is?

[00:05:51](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=351) David Wynn: Depending on how good your Googling or using GPT is, observability can be defined simply as logs, metrics, and traces. Essentially, an application generates data about itself to show how well it's running. Collecting and displaying that information is observability.

A broader way to define it is the process of ensuring that what you’re doing is operating as expected. This understanding goes beyond any specific metric, allowing for flexibility in approach. The observability space is transitioning because our data has become enormous, reaching hundreds of terabytes to petabytes generated daily.

When running applications on multiple machines, observability software pulls all that data together to find what’s important. However, scalability has become an issue as the volume increases.

[00:07:24](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=444) Justin Grammens: I've worked in the Internet of Things and sensors for over 15 years. At some point, processing becomes overwhelming, and I turned to machine learning and deep learning to filter through the noise. Can you discuss AI's role in observability?

[00:07:54](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=474) David Wynn: There’s a group of people very hopeful about AI, believing that if we throw all our data into it, it will find solutions. On the other hand, we know that errors and crashes are bad, and we generally understand the data we have. For example, at Edge Delta, we use a traditional machine learning model to identify statistical abnormalities. Then, we feed this information into an LLM to suggest next actions to take.

This is like having a 2 a.m. crib sheet for trouble-shooting—a quick list of things to check if something goes wrong. This process can be very useful for resolving issues and getting people back to bed, especially for those new to the team.

[00:09:27](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=567) Justin Grammens: It's a practical application of generative AI, particularly for junior engineers on call. They need backup when systems fail at inconvenient times.

[00:09:50](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=590) David Wynn: Exactly. When debugging, you may be isolating problems across teams and encountering new challenges. This complexity can vary greatly if there’s a lack of communication around team decision-making and implementation.

[00:10:21](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=621) Justin Grammens: I've talked to engineers who have difficulty deciphering logs and hardware dumps. They're looking for anomaly detection to guide them when they aren't familiar with the code or libraries.

[00:10:50](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=650) David Wynn: Different approaches exist to abstract problems. Some errors may require immediate resolution, while other conditions involve deeper awareness of a broader context. Higher-level issues may relate to architecture state or model performance.

LLMs can help translate command logs into actionable insights, but they struggle with new or unprecedented problems. For instance, I encountered a unique billing error at Intap—a failure in our sync service due to an unpaid account. LLMs might not provide guidance on rare errors that haven’t been encountered before, but they excel in generating ideas and constructing outlines for broader queries.

[00:13:55](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=835) Justin Grammens: I agree that LLMs lack intuition when it comes to errors. They're great for specific tasks, but they struggle with nuances and unique scenarios. And while they improve efficiency in mundane tasks, they're not perfect tools for summarizing or assessing the uniqueness of issues.

[00:15:47](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=947) David Wynn: Absolutely. While they can help streamline processes, their ability to resolve complex problems is limited.

[00:15:59](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=959) Justin Grammens: There’s concern around AI potentially replacing jobs. What’s your view on that?

[00:16:01](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=961) David Wynn: I’m generally not worried. If you try to get these technologies to perform tasks you’re concerned about, you might find challenges. For example, creating sophisticated art with AI can often lead to frustrating results. People will always seek specificity and control, which can be lost with AI.

My impression is that AI is not going to completely replace roles but will enhance processes, helping eliminate inefficient tasks. AI will streamline operations, allowing people to focus on higher-level problem-solving.

[00:18:53](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=1133) Justin Grammens: Makes sense. Current tools enhance capabilities rather than completely displace them. People will experience increased ROI with smart tools. 

[00:20:26](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=1226) David Wynn: Exactly! Many current AI tools become more intelligent through user interaction, enhancing daily workflows.

[00:20:40](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=1240) Justin Grammens: While generative AI has captured imaginations, many startups will fade away amidst the hype. Existing products will adapt and integrate AI efficiently.

[00:21:44](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=1304) David Wynn: Agreed! Many companies are focusing on value-driven models rather than sheer size. Innovations will improve specific applications, creating more effective and targeted solutions.

[00:22:59](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=1379) Justin Grammens: Observability is key to identifying and fixing system issues, as we saw with CrowdStrike recently. It highlights the need for better testing and pre-release verification of system changes.

[00:27:52](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=1672) David Wynn: Absolutely! If a deployment has a large blast radius, precautions should have been taken to catch potential issues. Observability measures must be effective enough to predict problems before they occur.

[00:30:50](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=1850) Justin Grammens: Drawing parallels with industry history—confidence can lead to overlooking vital checks.  Just as NASA faced challenges due to rapid launches, companies risk similar pitfalls without due diligence.

[00:31:39](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=1899) David Wynn: Yes, we must prevent overconfidence in testing and deploy agile systems that acknowledge potential issues.

[00:34:08](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=2048) Justin Grammens: Many companies fail to employ chaos engineering, disregarding the value of stress-testing systems to ensure resilience.

[00:34:24](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=2064) David Wynn: Indeed! Chaos engineering forces an organization to consider the worst-case scenarios actively while addressing system performance expectations.

[00:34:53](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=2093) Justin Grammens: As we wrap up, what advice would you give someone entering the technology field?

[00:35:04](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=2104) David Wynn: My advice is to follow your interests. The niche topics you’re drawn to may lead to innovative ideas. You can find resources on almost anything today. Engage with communities, ask questions online, fail, and learn—it's all part of the process!

[00:36:53](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=2213) Justin Grammens: Great advice. Even older resources can provide valuable insights, as industries often evolve at a slower pace.

[00:39:12](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=2352) David Wynn: Exactly! It’s about finding that intersection of interests and skills.

[00:40:00](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=2400) Justin Grammens: How can people connect with you after this conversation?

[00:40:21](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=2421) David Wynn: LinkedIn is best for reaching me. I have a professional-looking blazer in my profile picture!

[00:40:47](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=2447) Justin Grammens: We'll have your LinkedIn page and the relevant links in our show notes. Is there anything else you'd like to share?

[00:41:58](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=2518) David Wynn: I want to leave everyone with a hopeful thought. Many of us feel overwhelmed by the speed of change, thinking we don’t matter. Remember, the giants we stand on had the same doubts. They created impactful things, and so can you.

[00:42:45](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=2565) Justin Grammens: Inspiring indeed, David! It’s been a pleasure having you on the show. We appreciate your time and insights. Wishing you the best at Edge Delta and your future endeavors.

[00:42:49](https://appliedai.buzzsprout.com/1101152/episodes/16272078-david-wynn-striking-the-balance-between-ai-summarizing-and-human-insight?t=2569) AI Announcer: You've listened to another episode of the Conversations on Applied AI podcast. We hope you are eager to learn more about applying artificial intelligence and deep learning within your organization. You can visit us at [AppliedAI.mn](https://www.google.com/search?q=AppliedAI.mn) to keep up to date on our events and connect with our amazing community. Please don't hesitate to reach out to Justin at [AppliedAI.mn](https://www.google.com/search?q=AppliedAI.mn) if you are interested in participating in a future episode. Thank you for listening.

In [47]:
outline_prompt = """
Below is an example **prompt** you can give to an LLM. It walks the model through **(1)** examining the nature of the podcast, **(2)** generating an outline in a code block, and **(3)** rewriting the transcript with named-anchor headings that match the outline. Be sure to replace any placeholders (like `<<FULL TRANSCRIPT HERE>>`) with the actual transcript text.

---

## Prompt for the LLM

You are given a podcast transcript and some instructions. Please follow these steps:

1. **Analyze the Nature of the Podcast**
   - Read through the entire transcript to understand its main themes, flow, and the natural breaks (such as introductions, key topics, speaker transitions, wrap-up).

2. **Create an Outline**
   - Summarize the podcast’s structure as an outline.
   - Use a code fence labeled `outline` to present this outline.
   - Each major section in the outline should reflect logical segments of the conversation (e.g., “Introduction,” “Guest Background,” “Core Discussion,” “Future Outlook,” “Conclusion,” etc.).
   - Example of how to structure it:
     ```outline
     1. Introduction
       a. AI Voice Intro
       b. Host’s Welcome
     2. Guest Background
       ...
     ```

3. **Rewrite the Transcript with Headings**
   - Insert headings in the cleaned-up version of the transcript corresponding to each section of the outline.
   - For each heading, use **named anchors** with the format:
     ```
     # <span id="section-name">Section Title</span>
     ```
   - Wherever a reference to that section might appear in the text (like a table of contents or references), link to it using:
     ```
     [Section Title](#section-name)
     ```
   - Use a number of #'s to indicate the heading level, copying the structure of the outline.

4. **Return the Output in Two Parts**
   1. The **Outline** in a fenced code block labeled `outline`.
   2. The **Rewritten Transcript** with headings using named anchors.

5. **Use This Transcript**
   **Transcript (raw):**
   ```
   <<FULL TRANSCRIPT HERE>>
   ```

### Example Illustrations of Format

- **Outline Code Block**:
  ```outline
  1. [Introduction](#introduction)
    a. [Podcast overview](#podcast-overview)
    b. [Introduction of guest](#introduction-of-guest)
  2. [Key Discussion](#key-discussion)
    a. [Early career](#early-career)
    b. [Tech adoption](#tech-adoption)
  3. [Conclusion](#conclusion)
    a. [Final thoughts](#final-thoughts)
    b. [Outro](#outro)
  ```

- **Named Anchor Headings** in the rewritten transcript:
  ```transcript
  # <span id="introduction">Introduction</span>

  [00:00] Host: Welcome to the show! ...

  # <span id="key-discussion">Key Discussion</span>

  [00:05] Guest: So let's dive in ...
  ```

### Final Instructions

1. **Don’t insert extra commentary** in the final output—just produce the outline and the rewritten transcript.
2. Include ```outline ...``` and ```transcript ...``` code fences in your response.
3. Return your final answer as **valid Markdown**.
"""


In [48]:
def outline_transcript(text):
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": outline_prompt},
            {"role": "user", "content": text},
        ]
    )
    return resp.choices[0].message.content

def extract_code_blocks(text: str):
    pattern = re.compile(r'```([^\n]+)\n([\s\S]*?)```')
    blocks = []
    for tag, content in pattern.findall(text):
        blocks.append({"tag": tag.strip(), "content": content.strip()})
    return blocks

def extract_outline_and_transcript(text):
    blocks = extract_code_blocks(text)
    return "\n\n".join([block["content"] for block in blocks])


In [49]:
outlined_transcript = outline_transcript(linked_markdown)
formatted_outline = extract_outline_and_transcript(outlined_transcript)

display(Markdown(formatted_outline))

1. [Introduction](#introduction)
   a. [Opening Remarks](#opening-remarks)
   b. [Guest Introduction](#guest-introduction)
2. [David Wynn's Background](#david-wynns-background)
   a. [Early Life and Career](#early-life-and-career)
   b. [Transition into Technology](#transition-into-technology)
3. [Understanding Observability](#understanding-observability)
   a. [Definition and Importance](#definition-and-importance)
   b. [AI's Role in Observability](#ais-role-in-observability)
4. [Challenges with AI](#challenges-with-ai)
   a. [Limitations of Large Language Models](#limitations-of-large-language-models)
   b. [AI's Impact on Jobs](#ais-impact-on-jobs)
5. [Best Practices in Technology](#best-practices-in-technology)
   a. [Advice for Newcomers](#advice-for-newcomers)
   b. [Importance of Community Engagement](#importance-of-community-engagement)
6. [Closing Thoughts and Insights](#closing-thoughts-and-insights)
   a. [Personal Reflections](#personal-reflections)
   b. [Conclusion](#conclusion)

# <span id="introduction">Introduction</span>

[00:00:00] David Wynn: The mental model I like to give people is imagine a guy in a bar overheard 10,000 hours of two people next to him talking about motorcycles. He has never seen, driven, heard, or felt one. When you ask him a question about motorcycles, he’ll probably give you the right answer. Occasionally, he might compliment how your torque smells. It's not his fault. He's just trying to piece together everything he could possibly know from what he's heard. He's not really thinking it through. I think that model is key for people to grasp. As a result, summarizing—where I'm trusting someone to come up with the important stuff I need to know—feels a bit suspicious to me. If we called it shortening, that’d be different. Summarizing, I’m a little wary of.

[00:00:48] AI Announcer: Welcome to the Conversations on Applied AI podcast, where Justin Grammens and the team at Emerging Technologies North talk with experts in the fields of artificial intelligence and deep learning. In each episode, we cut through the hype and dive into how these technologies are being applied to real-world problems today. We hope you find this episode educational and applicable to your industry. Connect with us to learn more about our organization at [AppliedAI.mn](https://www.google.com/search?q=AppliedAI.mn). Enjoy.

# <span id="opening-remarks">Opening Remarks</span>

[00:01:19] Justin Grammens: Welcome, everyone, to the Conversations on Applied AI podcast. I'm your host, Justin Grammens, and our guest today is David Wynn. David is the Principal Solution Architect at EdgeDelta and an enthusiast for all things technology. 

With a career spanning 15 years, including a pivotal role at Google Cloud for Games, David has cultivated a deep understanding of technical sales, solution architecture, and the transformative power of cloud services in the gaming industry and beyond. His unique perspective challenges the overreliance on AI, advocating for a balanced approach that leverages human ingenuity alongside technological advancements, especially in areas like observability.

When he's not steering innovation, David revels in the vibrant geek culture of Atlanta and is a regular at DragonCon, the largest multimedia popular culture convention focusing on science fiction and fantasy. His diverse interests span from technology to philosophy. I’m sure we’re going to have an awesome conversation today about AI and its applications. Thank you so much for being on the program.

# <span id="guest-introduction">Guest Introduction</span>

[00:02:11] David Wynn: Absolutely, Justin. Pleasure to be here.

# <span id="david-wynns-background">David Wynn's Background</span>

# <span id="early-life-and-career">Early Life and Career</span>

[00:02:13] Justin Grammens: Awesome. I touched on some of your background at Google and your current role. I'm curious, how did you get into technology? Were you always fascinated with programming?

[00:02:31] David Wynn: I grew up in Greenville, South Carolina, which is not a technical hub in the U.S. My only teacher, Miss Harmon, was my computer science teacher throughout high school. That was just enough to spark my interest, but I went to college for economics instead. I had the wrong impression about what it meant to be a programmer, so I thought I’d do something else.

After college, I ended up spending time at UPS during the 2008 crash, where a PhD and I worked on econometrics to improve forecasting. My role involved a lot of VBA code to make Excel work for 250 time series data sets, which is where I started developing practical skills.

# <span id="transition-into-technology">Transition into Technology</span>

Later, I moved to California for a relationship, and at the Stanford career fair, I found a lot of technology opportunities. I ended up doing post-sales implementation middleware, which was a form of ETL. I built scalable data pipelines for law firms and transitioned into observability at Sumo Logic. Eventually, I joined Google Cloud when it was small, around 70 sales engineers globally, which was a rapid learning experience for cloud technologies. Now, I'm back in observability at Edge Delta.

# <span id="understanding-observability">Understanding Observability</span>

# <span id="definition-and-importance">Definition and Importance</span>

[00:05:42] Justin Grammens: Great journey! For our audience, could you define what observability is?

[00:05:51] David Wynn: Depending on how good your Googling or using GPT is, observability can be defined simply as logs, metrics, and traces. Essentially, an application generates data about itself to show how well it's running. Collecting and displaying that information is observability.

A broader way to define it is the process of ensuring that what you’re doing is operating as expected. This understanding goes beyond any specific metric, allowing for flexibility in approach. The observability space is transitioning because our data has become enormous, reaching hundreds of terabytes to petabytes generated daily.

When running applications on multiple machines, observability software pulls all that data together to find what’s important. However, scalability has become an issue as the volume increases.

# <span id="ais-role-in-observability">AI's Role in Observability</span>

[00:07:24] Justin Grammens: I've worked in the Internet of Things and sensors for over 15 years. At some point, processing becomes overwhelming, and I turned to machine learning and deep learning to filter through the noise. Can you discuss AI's role in observability?

[00:07:54] David Wynn: There’s a group of people very hopeful about AI, believing that if we throw all our data into it, it will find solutions. On the other hand, we know that errors and crashes are bad, and we generally understand the data we have. For example, at Edge Delta, we use a traditional machine learning model to identify statistical abnormalities. Then, we feed this information into an LLM to suggest next actions to take.

This is like having a 2 a.m. crib sheet for trouble-shooting—a quick list of things to check if something goes wrong. This process can be very useful for resolving issues and getting people back to bed, especially for those new to the team.

# <span id="challenges-with-ai">Challenges with AI</span>

# <span id="limitations-of-large-language-models">Limitations of Large Language Models</span>

[00:09:27] Justin Grammens: It's a practical application of generative AI, particularly for junior engineers on call. They need backup when systems fail at inconvenient times.

[00:09:50] David Wynn: Exactly. When debugging, you may be isolating problems across teams and encountering new challenges. This complexity can vary greatly if there’s a lack of communication around team decision-making and implementation.

[00:10:21] Justin Grammens: I've talked to engineers who have difficulty deciphering logs and hardware dumps. They're looking for anomaly detection to guide them when they aren't familiar with the code or libraries.

[00:10:50] David Wynn: Different approaches exist to abstract problems. Some errors may require immediate resolution, while other conditions involve deeper awareness of a broader context. Higher-level issues may relate to architecture state or model performance.

LLMs can help translate command logs into actionable insights, but they struggle with new or unprecedented problems. For instance, I encountered a unique billing error at Intap—a failure in our sync service due to an unpaid account. LLMs might not provide guidance on rare errors that haven’t been encountered before, but they excel in generating ideas and constructing outlines for broader queries.

# <span id="ais-impact-on-jobs">AI's Impact on Jobs</span>

[00:13:55] Justin Grammens: I agree that LLMs lack intuition when it comes to errors. They're great for specific tasks, but they struggle with nuances and unique scenarios. And while they improve efficiency in mundane tasks, they're not perfect tools for summarizing or assessing the uniqueness of issues.

[00:15:47] David Wynn: Absolutely. While they can help streamline processes, their ability to resolve complex problems is limited.

[00:15:59] Justin Grammens: There’s concern around AI potentially replacing jobs. What’s your view on that?

[00:16:01] David Wynn: I’m generally not worried. If you try to get these technologies to perform tasks you’re concerned about, you might find challenges. For example, creating sophisticated art with AI can often lead to frustrating results. People will always seek specificity and control, which can be lost with AI.

My impression is that AI is not going to completely replace roles but will enhance processes, helping eliminate inefficient tasks. AI will streamline operations, allowing people to focus on higher-level problem-solving.

# <span id="best-practices-in-technology">Best Practices in Technology</span>

# <span id="advice-for-newcomers">Advice for Newcomers</span>

[00:18:53] Justin Grammens: Makes sense. Current tools enhance capabilities rather than completely displace them. People will experience increased ROI with smart tools. 

[00:20:26] David Wynn: Exactly! Many current AI tools become more intelligent through user interaction, enhancing daily workflows.

[00:20:40] Justin Grammens: While generative AI has captured imaginations, many startups will fade away amidst the hype. Existing products will adapt and integrate AI efficiently.

[00:21:44] David Wynn: Agreed! Many companies are focusing on value-driven models rather than sheer size. Innovations will improve specific applications, creating more effective and targeted solutions.

# <span id="importance-of-community-engagement">Importance of Community Engagement</span>

[00:22:59] Justin Grammens: Observability is key to identifying and fixing system issues, as we saw with CrowdStrike recently. It highlights the need for better testing and pre-release verification of system changes.

[00:27:52] David Wynn: Absolutely! If a deployment has a large blast radius, precautions should have been taken to catch potential issues. Observability measures must be effective enough to predict problems before they occur.

[00:30:50] Justin Grammens: Drawing parallels with industry history—confidence can lead to overlooking vital checks. Just as NASA faced challenges due to rapid launches, companies risk similar pitfalls without due diligence.

[00:31:39] David Wynn: Yes, we must prevent overconfidence in testing and deploy agile systems that acknowledge potential issues.

[00:34:08] Justin Grammens: Many companies fail to employ chaos engineering, disregarding the value of stress-testing systems to ensure resilience.

[00:34:24] David Wynn: Indeed! Chaos engineering forces an organization to consider the worst-case scenarios actively while addressing system performance expectations.

# <span id="closing-thoughts-and-insights">Closing Thoughts and Insights</span>

# <span id="personal-reflections">Personal Reflections</span>

[00:34:53] Justin Grammens: As we wrap up, what advice would you give someone entering the technology field?

[00:35:04] David Wynn: My advice is to follow your interests. The niche topics you’re drawn to may lead to innovative ideas. You can find resources on almost anything today. Engage with communities, ask questions online, fail, and learn—it’s all part of the process!

[00:36:53] Justin Grammens: Great advice. Even older resources can provide valuable insights, as industries often evolve at a slower pace.

[00:39:12] David Wynn: Exactly! It’s about finding that intersection of interests and skills.

# <span id="conclusion">Conclusion</span>

[00:40:00] Justin Grammens: How can people connect with you after this conversation?

[00:40:21] David Wynn: LinkedIn is best for reaching me. I have a professional-looking blazer in my profile picture!

[00:40:47] Justin Grammens: We'll have your LinkedIn page and the relevant links in our show notes. Is there anything else you'd like to share?

[00:41:58] David Wynn: I want to leave everyone with a hopeful thought. Many of us feel overwhelmed by the speed of change, thinking we don’t matter. Remember, the giants we stand on had the same doubts. They created impactful things, and so can you.

[00:42:45] Justin Grammens: Inspiring indeed, David! It’s been a pleasure having you on the show. We appreciate your time and insights. Wishing you the best at Edge Delta and your future endeavors.

[00:42:49] AI Announcer: You've listened to another episode of the Conversations on Applied AI podcast. We hope you are eager to learn more about applying artificial intelligence and deep learning within your organization. You can visit us at [AppliedAI.mn](https://www.google.com/search?q=AppliedAI.mn) to keep up to date on our events and connect with our amazing community. Please don't hesitate to reach out to Justin at [AppliedAI.mn](https://www.google.com/search?q=AppliedAI.mn) if you are interested in participating in a future episode. Thank you for listening.